In [1]:
import pandas
import os

In [2]:
cells_of_interest=[x.split() for x in """20042_D4	DarkRed
18255_E8	DarkRed
18254_D2	DarkRed
18252_B12	DarkRed
20042_D2	DarkRed
18312_B4	DarkRed
18315_E6	DarkRed
18255_E7	DarkRed
20036_D11	DarkRed
19915_B9	DarkRed
20034_B6	DarkRed
20031_F8	green
17327_A4	green
18048_G8	green
18044_C6	green
18042_A7	green
18043_B2	green
17329_C10	green
17333_G6	green
18088_G2	green
18044_C5	green
18262_E11	green
18262_E2	green
18258_A7	green
17327_A2	green
17329_C10	green""".split('\n')]
cells_of_interest = [x[0] for x in cells_of_interest]

In [3]:
def is_cell_present(cells_of_interest, path):
    for cell in cells_of_interest:
        if cell in path:
            return True
    return False

In [4]:
with open('merge_bw_all.sh') as instream:
    cell_ids = set()
    uniq_bw = {}
    all_bw = {}
    for line in instream:
        args = line.rstrip().split(' ')[3:]
        cluster = args[0]
        for pathname in args[1:]:
            path, name = os.path.split(pathname)
            cell_id = name[:name.find('mm10')-1]
            if cell_id in cells_of_interest:
                cell_ids.add(cell_id)
                if name.endswith('_all.bw'):
                    all_bw.setdefault(cluster, []).append(pathname)
                elif name.endswith('_uniq.bw'):
                    uniq_bw.setdefault(cluster, []).append(pathname)
                else:
                    print(pathname)
                


In [5]:
sum([len(uniq_bw[k]) for k in uniq_bw])

25

In [6]:
set(cells_of_interest).difference(cell_ids)

set()

In [7]:
cell_ids.difference(cells_of_interest)

set()

In [8]:
len(set(cell_ids))

25

In [9]:
assert len(set(cell_ids)) == sum([len(uniq_bw[k]) for k in uniq_bw])

In [10]:
target = 'brian_subset_201905291133'

In [11]:
with open('merge_bw_{target}.sh'.format(target=target), 'wt') as outstream:
    for group in [uniq_bw, all_bw]:
        for cluster_name in group:
            args = ['python3', '../merge_bw.py', '-o', cluster_name]
            args.extend(group[cluster_name])
            outstream.write(' '.join(args))
            outstream.write(os.linesep)

In [12]:
os.getcwd()

'/woldlab/loxcyc/home/diane/proj/C1_mouse_limb_combined'

In [13]:
os.path.join(target, target+'.hub.txt')

'brian_subset_201905291133/brian_subset_201905291133.hub.txt'

In [14]:
hub="""hub hub
shortLabel {target}
longLabel {target}
genomesFile {target}.genomes.txt
email diane@caltech.edu
""".format(target=target)

with open(os.path.join(target, target+'.hub.txt'), 'wt') as outstream:
    outstream.write(hub)

In [15]:
genome="""genome mm10
trackDb mm10/trackDb-{target}.txt
""".format(target=target)

with open(os.path.join(target, target+'.genomes.txt'), 'wt') as outstream:
    outstream.write(genome)

In [16]:
trackDb = """track composite
shortLabel bigwigs
longLabel bigwigs
type bigWig
visibility full
dimensions dimX=multiread
sortOrder multiread
subGroup1 view Views signal=signal
subGroup2 multiread multiread all=all_reads uniq=unique_only
compositeTrack on

    track signal
    shortLabel signal
    longLabel signal
    type bigWig
    visibility full
    parent composite on
    view signal

            track greenall
            bigDataUrl http://woldlab.caltech.edu/~diane/C1_mouse_limb_combined/{target}/green-mm10-M4-male_all.bw
            shortLabel greenall
            longLabel Muscle 1 all
            type bigWig
            visibility full
            color 0,255,0
            subGroups multiread=all view=signal
            parent signal on
    
            track greenuniq
            bigDataUrl http://woldlab.caltech.edu/~diane/C1_mouse_limb_combined/{target}/green-mm10-M4-male_uniq.bw
            shortLabel greenuniq
            longLabel Muscle 1 uniq
            type bigWig
            visibility full
            color 0,255,0
            subGroups multiread=uniq view=signal
            parent signal on

            track DarkRedall
            bigDataUrl http://woldlab.caltech.edu/~diane/C1_mouse_limb_combined/{target}/DarkRed-mm10-M4-male_all.bw
            shortLabel DarkRedall
            longLabel Muscle 3 all
            type bigWig
            visibility full
            color 191,0,0
            subGroups multiread=all view=signal
            parent signal on
    
            track DarkReduniq
            bigDataUrl http://woldlab.caltech.edu/~diane/C1_mouse_limb_combined/{target}/DarkRed-mm10-M4-male_uniq.bw
            shortLabel DarkReduniq
            longLabel Muscle 3 uniq
            type bigWig
            visibility full
            color 191,0,0
            subGroups multiread=uniq view=signal
            parent signal on
""".format(target=target)

with open(os.path.join(target, 'mm10', 'trackDb-' + target +'.txt'), 'wt') as outstream:
    outstream.write(trackDb)

In [17]:
cmd="hubCheck http://woldlab.caltech.edu/~diane/C1_mouse_limb_combined/{target}/{target}.hub.txt".format(target=target)
print(cmd)

hubCheck http://woldlab.caltech.edu/~diane/C1_mouse_limb_combined/brian_subset_201905291133/brian_subset_201905291133.hub.txt
